In [73]:
import pandas as pd
import json
import joblib

file = open('NBA Team Data.json')
teams = json.load(file)
current_nba_data = pd.read_csv('../Data/Current NBA Game Data.csv', index_col=0)

## Getting Prediction Stats
Run Current Data Scraper.ipynb get the most up to date stats for all teams. 

The program will then process the data, feed it to the machine learning model, and make a prediction!

In [74]:
# retrieves and formats recent NBA game data into a pandas data frame
def get_prediction_stats(away_team, home_team):
    away_team_data = current_nba_data.loc[teams[away_team]]
    away_team_data.index = [f'{i}_A' for i in list(away_team_data.index)]
    home_team_data = current_nba_data.loc[teams[home_team]]
    home_team_data.index = [f'{i}_H' for i in list(home_team_data.index)]

    game_prediction_data = pd.concat([away_team_data, home_team_data])
    game_prediction_data['NEXT_HOME/AWAY'] = 0.0
    game_prediction_data['AWAY_TEAM'] = away_team
    game_prediction_data['HOME_TEAM'] = home_team


    return pd.DataFrame(game_prediction_data).T

In [75]:
def make_prediction(away_team, home_team):    
    game_prediction_data = get_prediction_stats(away_team, home_team)

    # loads the saved preprocessing pipeline and ml model
    scaler, rr, training_values, target = joblib.load('Ridge Classifier Model.pkl')

    # applies the preprocessing pipeline to current game prediction data and make a prediciton
    scaled_data = pd.DataFrame(scaler.transform(game_prediction_data[training_values]), columns=training_values)
    game_prediction_data["PREDICTED_WINNER"] = rr.predict(scaled_data)

    # formats and prints prediction for current game
    win_team = away_team if game_prediction_data["PREDICTED_WINNER"][0] == 1.0 else home_team
    print(f'{away_team} at {home_team}    Predicted Winner : {win_team}')

    return game_prediction_data

## Making a Prediction!
Run make_prediction with team abbreviations. 
The rr model will then print it's prediction for that given game.

NOTE : Make sure you specify which teams is home and which is away to increase the accuracy of the prediction 

In [76]:
make_prediction(away_team='DET', home_team='MIN')

MIN at DET : MIN


,PTS_AVG_A,PTS_A,PTS_OPP_AVG_A,PTS_OPP_A,WIN/LOSS_AVG_A,WIN/LOSS_A,FG_AVG_A,FG_A,FGA_AVG_A,FGA_A,...,PF_OPP_H,TRB_H,TRB_AVG_H,TRB_OPP_H,TRB_OPP_AVG_H,HOME/AWAY_H,NEXT_HOME/AWAY,AWAY_TEAM,HOME_TEAM,PREDICTED_WINNER
0,111.3,106.0,112.7,129.0,0.667,0.0,40.0,34.0,82.3,73.0,...,22.0,52.0,43.0,42.0,41.4,1.0,0.0,MIN,DET,1.0
